In [ ]:
%load_ext autoreload

In [ ]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from shapely import geometry, ops
from shapely.geometry import MultiLineString, LineString, Point
import os
import concurrent.futures
from shapely.ops import unary_union
import xyt

import time

from panel_functions import *

In [ ]:
%%time
# Définir le CRS du projet (EPSG:4326 for WGS84)
target_crs = 'EPSG:4326'
print("CRS du projet: WGS84 \n")

#Charger les étapes
legs = pd.read_pickle('../Data/dumps_motiontag/storyline_time_space_filters/legs.pkl')
print("Fichier étape chargé")

#Charger les activités
staypoints = pd.read_pickle('../Data/dumps_motiontag/storyline_formated/staypoints.pkl').reset_index(drop=True)
print("Fichier activité chargé")

#Charger les user_statistics
usr_stats = pd.read_pickle('../Data/processed_feuille_de_route/gps_user_statistics.pkl')
print("Fichier statistiques utilisateur·ices chargé")

In [ ]:
list_ge = usr_stats.loc[usr_stats.KT_home_survey == 'GE', 'user_id_fors'].to_list()

In [ ]:
legs_ge = legs.loc[legs.user_id_fors.isin(list_ge)]

In [ ]:
legs_ge[['user_id_fors','mode','low_quality_legs_1','low_quality_legs_2','length_leg','geometry']]

In [ ]:
legs_ge[['user_id_fors','mode','low_quality_legs_1','low_quality_legs_2','length_leg','geometry']].to_file('../Data/temp_files/extract_legs_GE.geojson', driver="GeoJSON")  

In [ ]:
legs_ge[['user_id_fors','mode','low_quality_legs_1','low_quality_legs_2','length_leg','geometry']].sample(frac=0.5)

In [ ]:
legs_ge['mode'].value_counts()

In [ ]:
#car
legs_ge[['user_id_fors','mode','low_quality_legs_1','low_quality_legs_2','length_leg','geometry']].loc[legs_ge['mode'] == 'Mode::Car'].to_file('../Data/temp_files/extract_legs_GE_car.geojson', driver="GeoJSON")  
#Bus
legs_ge[['user_id_fors','mode','low_quality_legs_1','low_quality_legs_2','length_leg','geometry']].loc[legs_ge['mode'] == 'Mode::Bus'].to_file('../Data/temp_files/extract_legs_GE_bus.geojson', driver="GeoJSON")  
#Walk
legs_ge[['user_id_fors','mode','low_quality_legs_1','low_quality_legs_2','length_leg','geometry']].loc[legs_ge['mode'] == 'Mode::Walk'].to_file('../Data/temp_files/extract_legs_GE_walk.geojson', driver="GeoJSON")


In [ ]:
#Train
legs_ge[['user_id_fors','mode','low_quality_legs_1','low_quality_legs_2','length_leg','geometry']].loc[legs_ge['mode'] == 'Mode::Train'].to_file('../Data/temp_files/extract_legs_GE_train.geojson', driver="GeoJSON")  

In [ ]:
#car_half
legs_ge[['user_id_fors','mode','low_quality_legs_1','low_quality_legs_2','length_leg','geometry']].loc[legs_ge['mode'] == 'Mode::Car'].sample(frac=0.5).to_file('../Data/temp_files/extract_legs_GE_car_sample.geojson', driver="GeoJSON")  


In [ ]:
#tram
legs_ge[['user_id_fors','mode','low_quality_legs_1','low_quality_legs_2','length_leg','geometry']].loc[legs_ge['mode'] == 'Mode::Tram'].sample(frac=0.5).to_file('../Data/temp_files/extract_legs_GE_tram.geojson', driver="GeoJSON")  


In [ ]:
#bike
legs_ge[['user_id_fors','mode','low_quality_legs_1','low_quality_legs_2','length_leg','geometry']].loc[legs_ge['mode'] == 'Mode::Bicycle'].sample(frac=0.5).to_file('../Data/temp_files/extract_legs_GE_bicycle.geojson', driver="GeoJSON")  


###  Export emails for interviews

In [ ]:
emails = pd.read_csv('../Data/dumps_motiontag/UserStatistics.EPFL-Panel.2023-04-24--2023-06-05.csv', sep=';')
emails.columns

In [ ]:
pop1_fors = ['CH5950','FR11511','CH18644','CH20781','CH25241','CH27556','CH5729','CH8471','FR10651','FR11102','FR13137','FR16651','FR18721','FR3507','FR480','FR5908','FR8384']
pop2_fors = ['CH13829','CH17367','CH18531','CH20900','CH21398','CH24231','CH28825','CH28876','CH29072','CH29325','CH8750','FR14670','FR16458','CH19422','FR8339','CH10352','CH1047','CH10757','CH10853','CH10946','CH10948','CH11047','CH1113','CH11143','CH11165','CH11210','CH11374','CH11524','CH11618','CH1185','CH12190','CH12283','CH12547','CH12811','CH12829','CH12966','CH13237','CH13286','CH135','CH13709','CH13898','CH14130','CH14199','CH14404','CH1442','CH14523','CH14602','CH14659','CH14837','CH14990','CH15033','CH15299','CH15389','CH15539','CH15547','CH15551','CH15576','CH16941','CH17146','CH17446','CH17560','CH17736','CH17809','CH17915','CH17936','CH17945','CH18055','CH18306','CH18341','CH18633','CH18636','CH19275','CH19302','CH19410','CH19731','CH19734','CH19787','CH20338','CH20382','CH20390','CH2050','CH20523','CH20743','CH2133','CH21469','CH21571','CH21597','CH22190','CH2259','CH22752','CH23031','CH23096','CH2316','CH23182','CH23224','CH23382','CH23483','CH23516','CH23954','CH24039','CH24061','CH24432','CH2470','CH25390','CH25406','CH2548','CH25732','CH26161','CH26464','CH2703','CH27659','CH27678','CH27846','CH27883','CH28043','CH28060','CH28179','CH28194','CH28257','CH28319','CH28340','CH28358','CH28367','CH28385','CH28388','CH28429','CH28658','CH28763','CH28959','CH28974','CH29098','CH29099','CH29131','CH29208','CH29269','CH29274','CH29318','CH29651','CH29723','CH29795','CH29813','CH29928','CH29934','CH3620','CH3773','CH3828','CH4353','CH4702','CH5165','CH5241','CH531','CH5419','CH5431','CH5513','CH555','CH5605','CH5612','CH5625','CH5637','CH5638','CH5639','CH5647','CH565','CH5816','CH6000','CH6277','CH683','CH6937','CH7053','CH7108','CH8345','CH8660','CH8806','CH8946','CH9050','CH9058','CH9136','CH9378','CH9381','CH9384','CH9458','CH9591','FR10050','FR10892','FR11198','FR11336','FR11484','FR11565','FR11907','FR12971','FR13032','FR1337','FR13454','FR13470','FR13921','FR14344','FR14798','FR14899','FR15397','FR15503','FR15629','FR15947','FR16338','FR16742','FR16785','FR16983','FR17016','FR17019','FR17112','FR17125','FR17795','FR17857','FR17883','FR18012','FR18347','FR18511','FR2473','FR2513','FR2573','FR2843','FR2915','FR2980','FR3150','FR3176','FR3283','FR3399','FR3410','FR3701','FR4040','FR4350','FR4709','FR4973','FR5132','FR5682','FR5918','FR6235','FR6350','FR6542','FR6589','FR6592','FR6840','FR689','FR6890','FR6953','FR6991','FR707','FR7359','FR7444','FR7642','FR8144','FR8452','FR9626']
pop_fors= ['CH10352','CH1047','CH10757','CH10853','CH10946','CH10948','CH11047','CH1113','CH11143','CH11165','CH11210','CH11374','CH11524','CH11618','CH1185','CH12190','CH12283','CH12547','CH12811','CH12829','CH12966','CH13237','CH13286','CH135','CH13709','CH13898','CH14130','CH14199','CH14404','CH1442','CH14523','CH14602','CH14659','CH14837','CH14990','CH15033','CH15299','CH15389','CH15539','CH15547','CH15551','CH15576','CH16941','CH17146','CH17446','CH17560','CH17736','CH17809','CH17915','CH17936','CH17945','CH18055','CH18306','CH18341','CH18633','CH18636','CH19275','CH19302','CH19410','CH19731','CH19734','CH19787','CH20338','CH20382','CH20390','CH2050','CH20523','CH20743','CH2133','CH21469','CH21571','CH21597','CH22190','CH2259','CH22752','CH23031','CH23096','CH2316','CH23182','CH23224','CH23382','CH23483','CH23516','CH23954','CH24039','CH24061','CH24432','CH2470','CH25390','CH25406','CH2548','CH25732','CH26161','CH26464','CH2703','CH27659','CH27678','CH27846','CH27883','CH28043','CH28060','CH28179','CH28194','CH28257','CH28319','CH28340','CH28358','CH28367','CH28385','CH28388','CH28429','CH28658','CH28763','CH28959','CH28974','CH29098','CH29099','CH29131','CH29208','CH29269','CH29274','CH29318','CH29651','CH29723','CH29795','CH29813','CH29928','CH29934','CH3620','CH3773','CH3828','CH4353','CH4702','CH5165','CH5241','CH531','CH5419','CH5431','CH5513','CH555','CH5605','CH5612','CH5625','CH5637','CH5638','CH5639','CH5647','CH565','CH5816','CH6000','CH6277','CH683','CH6937','CH7053','CH7108','CH8345','CH8660','CH8806','CH8946','CH9050','CH9058','CH9136','CH9378','CH9381','CH9384','CH9458','CH9591','FR10050','FR10892','FR11198','FR11336','FR11484','FR11565','FR11907','FR12971','FR13032','FR1337','FR13454','FR13470','FR13921','FR14344','FR14798','FR14899','FR15397','FR15503','FR15629','FR15947','FR16338','FR16742','FR16785','FR16983','FR17016','FR17019','FR17112','FR17125','FR17795','FR17857','FR17883','FR18012','FR18347','FR18511','FR2473','FR2513','FR2573','FR2843','FR2915','FR2980','FR3150','FR3176','FR3283','FR3399','FR3410','FR3701','FR4040','FR4350','FR4709','FR4973','FR5132','FR5682','FR5918','FR6235','FR6350','FR6542','FR6589','FR6592','FR6840','FR689','FR6890','FR6953','FR6991','FR707','FR7359','FR7444','FR7642','FR8144','FR8452','FR9626','CH5950','FR11511']

In [ ]:
pop1_motiontag = usr_stats.loc[usr_stats.user_id_fors.isin(pop1_fors), 'user_id_motiontag'].to_list()
pop2_motiontag = usr_stats.loc[usr_stats.user_id_fors.isin(pop2_fors), 'user_id_motiontag'].to_list()
pop_motiontag = usr_stats.loc[usr_stats.user_id_fors.isin(pop_fors), 'user_id_motiontag'].to_list()

In [ ]:
pop1_email = emails.loc[emails['id'].isin(pop1_motiontag),['email', 'id']].merge(usr_stats[['user_id_fors','user_id_motiontag']], left_on='id',right_on='user_id_motiontag')[['email','user_id_fors']]
pop2_email = emails.loc[emails['id'].isin(pop2_motiontag),['email', 'id']].merge(usr_stats[['user_id_fors','user_id_motiontag']], left_on='id',right_on='user_id_motiontag')[['email','user_id_fors']]
pop_email = emails.loc[emails['id'].isin(pop_motiontag),['email', 'id']].merge(usr_stats[['user_id_fors','user_id_motiontag']], left_on='id',right_on='user_id_motiontag')[['email','user_id_fors']]

In [ ]:
pop1_email.to_csv('../Data/temp_files/pop1_emails.csv')
pop2_email.to_csv('../Data/temp_files/pop2_emails.csv')
pop_email.to_csv('../Data/temp_files/pop_emails.csv', sep=';')

In [ ]:
len(pop2_fors)